In [0]:
import requests
import json

In [0]:
# Secret is already created. The following is getting the secret
token = dbutils.secrets.get(scope="databricks-secrets", key="db_token")
space_id = '01f0bde9644518fb8a6b56936b08f2bf'

##Get the details of the Genie Space

This retrives the full details of the Genie Space. 


This is used to inspect the configurations, debugging, and versioning

In [0]:
#Get the details of the Genie space
def get_genie_space(token, space_id):
    url = f'https://dbc-527bf15c-60b6.cloud.databricks.com//api/2.0/genie/spaces/{space_id}'
    headers = {"Authorization": "Bearer " + token,
        "Content-Type": "application/json"}
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        try:
            # print the JSON response
            data = response.json()
            print(json.dumps(data, indent=4))
        except Exception as e:
            print("Response was 200 but not valid JSON")
            print("Exception:", str(e))
            print("Raw text:", response.text)
    else: # Result was not 200
        print("Error:", response.status_code)
        print("Response:", response.text)

    return 


## Update Genie Space

This api can be used to update infromation about the genie space

In [0]:
# PATCH the Genie space
def update_genie_space(token, space_id, description=None, title=None, warehouse_id=None, serialized_space=None):
    url = f"https://dbc-527bf15c-60b6.cloud.databricks.com/api/2.0/genie/spaces/{space_id}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    # Building the body  dynamically
    body = {}
    if description:
        body["description"] = description
    if title:
        body["title"] = title
    if warehouse_id:
        body["warehouse_id"] = warehouse_id
    if serialized_space:
        body["serialized_space"] = serialized_space

    response = requests.patch(url, headers=headers, json=body)

    if response.status_code == 200:
        try:
            # print the JSON response
            data = response.json()
            print(json.dumps(data, indent=4))
            return data
        except Exception as e:
            print("Response was 200 but not valid JSON")
            print("Exception:", str(e))
            print("Raw text:", response.text)
    else:
        print("Error:", response.status_code)
        print("Response:", response.text)
        return None
    
    return




## Listing conversations in the genie space

This API lists all of the conversations.

In [0]:
# Get all of the conversations in the Genie Space
def get_genie_space_conversations(token, space_id, page_size=20, page_token=None, include_all=False):
    url = f"https://dbc-527bf15c-60b6.cloud.databricks.com/api/2.0/genie/spaces/{space_id}/conversations"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    # The query parameters
    params = {
        "page_size": page_size,
        "include_all": str(include_all).lower() 
    }
    if page_token:
        params["page_token"] = page_token

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        try:
            data = response.json()
            print(json.dumps(data, indent=4))
            return data
        except Exception as e:
            print("Response was 200 but not valid JSON")
            print("Exception:", str(e))
            print("Raw text:", response.text)
            return None
    else:
        print("Error:", response.status_code)
        print("Response:", response.text)
        return None




## Delete Genie Space Conversation


Removes a specific conversation from the Genie Space. This is useful for cleaning up old or irrelevant conversations.




In [0]:
# Delete a specific conversation in a Genie Space
def delete_genie_space_conversation(token, space_id, conversation_id):
    url = f"https://dbc-527bf15c-60b6.cloud.databricks.com/api/2.0/genie/spaces/{space_id}/conversations/{conversation_id}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    response = requests.delete(url, headers=headers)

    if response.status_code == 200 or response.status_code == 204:
        print(f"Conversation {conversation_id} deleted successfully.")
        return True
    else:
        print("Error:", response.status_code)
        print("Response:", response.text)
        return False




##Create a conversation message
This will create a message

In [0]:
# Create a new message in a Genie Space conversation
def create_genie_message(token, space_id, conversation_id, content):
    url = f"https://dbc-527bf15c-60b6.cloud.databricks.com/api/2.0/genie/spaces/{space_id}/conversations/{conversation_id}/messages"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    body = {
        "content": content
    }

    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        try:
            data = response.json()
            print(json.dumps(data, indent=4))
            return data
        except Exception as e:
            print("Response was 200 but not valid JSON")
            print("Exception:", str(e))
            print("Raw text:", response.text)
            return None
    else:
        print("Error:", response.status_code)
        print("Response:", response.text)
        return None


## Send message feedback
This will send feedback about the message
need to have the message id

In [0]:
def send_genie_message_feedback(token, space_id, conversation_id, message_id, rating):
    url = f"https://dbc-527bf15c-60b6.cloud.databricks.com/api/2.0/genie/spaces/{space_id}/conversations/{conversation_id}/messages/{message_id}/feedback"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    body = {
        "rating": rating
    }

    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        try:
            data = response.json()
            print(json.dumps(data, indent=4))
            return data
        except Exception as e:
            print("Response was 200 but not valid JSON")
            print("Exception:", str(e))
            print("Raw text:", response.text)
            return None
    else:
        print("Error:", response.status_code)
        print("Response:", response.text)
        return None

## Workflow
The following is an example of a possible workflow

Any of the functions can be independently called as well

In [0]:

def genie_workflow(token, space_id):
    #Getting the genie space details
    print("Fetching Genie space details")
    get_genie_space(token, space_id)

    # listing the conversations
    print("\nListing conversations")
    conversations = get_genie_space_conversations(token, space_id)
    if not conversations or "conversations" not in conversations:
        print("No conversations found")
        return
    
    #Pick the first conversation
    conversation_id = conversations["conversations"][0]["id"]
    print(f"Using conversation: {conversation_id}")

    #Creating a new message (This message can be changed)
    message_content = f"What is the best well in the last 12 months."
    print("\nSending message")
    message = create_genie_message(token, space_id, conversation_id, message_content)
    if not message:
        print("Message creation failed")
        return
    
    message_id = message["id"]
    print(f"Message sent with ID: {message_id}")

    # Send feedback on the message to tell genie if its correct or not
    print("\nSending feedback")
    feedback = send_genie_message_feedback(token, space_id, conversation_id, message_id, rating="positive")
    if feedback:
        print("Feedback submitted successfully")


genie_workflow(token, space_id)